In [15]:
from hotels50k import Hotels50KDataset, UseOriginalTestSplitManager
from powerful_benchmarker.split_managers import IndexSplitManager
from torchvision import transforms
from torchvision import datasets
import torch
import numpy as np

In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
dataset = Hotels50KDataset(root='/data/thesis/Hotels-50K', target='chains', download=True)

Cloning https://github.com/GWUvision/Hotels-50K into /data/thesis/Hotels-50K
Repo already cloned
Unpacking dataset archive
Archive already unpacked
Train images already loaded
Archive already downloaded
Archive already unpacked
Creating symlinks
Creating LMDB image folders
Creating LMDB file for target chains and split test
Loading dataset from /data/thesis/Hotels-50K/symlinks/test/chains
Generate LMDB to /data/thesis/Hotels-50K/lmdb/chains/test.lmdb
2973 2973


100%|██████████| 2973/2973 [00:01<00:00, 1915.10it/s]


Flushing database ...
Creating LMDB file for target chains and split train
Loading dataset from /data/thesis/Hotels-50K/symlinks/train/chains
Generate LMDB to /data/thesis/Hotels-50K/lmdb/chains/train.lmdb
9219 9219


100%|██████████| 9219/9219 [00:17<00:00, 529.06it/s] 


Flushing database ...
Creating LMDB file for target hotels and split test
Loading dataset from /data/thesis/Hotels-50K/symlinks/test/hotels
Generate LMDB to /data/thesis/Hotels-50K/lmdb/hotels/test.lmdb
16169 16169


 13%|█▎        | 2139/16169 [00:08<00:58, 238.79it/s]


KeyboardInterrupt: 

In [8]:
dataset.labels.shape

(10041,)

In [9]:
len(dataset.original_train_dataset.targets), len(dataset.original_test_dataset.targets)

(7068, 2973)

In [10]:
import numpy as np

In [11]:
np.where(dataset.labels==1)

(array([7547, 7548, 7549, ..., 8595, 8596, 8597]),)

In [13]:
assert list(dataset.labels[0:100]) == [dataset[i][1] for i in range(0, 100)]

In [14]:
ds = dataset

# first level of dictionary is transform type
# second level consists of separate dataset objects for each split
datasets = {"eval": {"train": ds,
                    "val": ds,
                    "test": ds}
            }

helper_split_manager = UseOriginalTestSplitManager()
split_manager = IndexSplitManager(num_training_partitions=4, num_training_sets=4, helper_split_manager=helper_split_manager)

split_manager.create_split_schemes(datasets)

test_set = split_manager.get_dataset("eval", "test")

for name in split_manager.split_scheme_names:
    split_manager.set_curr_split_scheme(name)
    print("CURR SPLIT SCHEME =", split_manager.curr_split_scheme_name)
    print("train len", len(split_manager.get_dataset("eval", "train")))
    print("val len", len(split_manager.get_dataset("eval", "val")))
    print("test len", len(split_manager.get_dataset("eval", "test")))

    print("train labels", split_manager.get_label_set("eval", "train"))
    print("val labels", split_manager.get_label_set("eval", "val"))
    print("test labels", split_manager.get_label_set("eval", "test"))
    
    assert set(split_manager.get_dataset("eval", "train").indices).intersection(split_manager.get_dataset("eval", "val").indices) == set()
    assert set(split_manager.get_dataset("eval", "train").indices).intersection(split_manager.get_dataset("eval", "test").indices) == set()
    assert set(split_manager.get_dataset("eval", "val").indices).intersection(split_manager.get_dataset("eval", "test").indices) == set()


CURR SPLIT SCHEME = UsingOriginalTest_Partitions4_0
train len 5301
val len 1767
test len 2973
train labels {0}
val labels {0}
test labels {0, 1, 2}
CURR SPLIT SCHEME = UsingOriginalTest_Partitions4_1
train len 5301
val len 1767
test len 2973
train labels {0}
val labels {0}
test labels {0, 1, 2}
CURR SPLIT SCHEME = UsingOriginalTest_Partitions4_2
train len 5301
val len 1767
test len 2973
train labels {0}
val labels {0}
test labels {0, 1, 2}
CURR SPLIT SCHEME = UsingOriginalTest_Partitions4_3
train len 5301
val len 1767
test len 2973
train labels {0}
val labels {0}
test labels {0, 1, 2}
